In [1]:
from mlx_lm import load, generate

model, tokenizer = load("mlx-community/phi-4-8bit")

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
from wla import WordLevelAligner

wla = WordLevelAligner(model_path=(model, tokenizer))

In [3]:
from collections import defaultdict

all_results = defaultdict(list)

In [4]:
examples = wla.get_examples()

In [5]:
test = examples[100:]
examples = examples[:100]

_all = [wla.fill_prompt(examples, t, 3) for t in test[:20]]
all_prompts = [a[0] for a in _all]
all_correct = [a[1] for a in _all]

In [6]:
from wla import Evaluator

evaluator = Evaluator()

In [7]:
def evaluate(prompt, correct):
    messages = [
        {'role':'user', 'content':prompt}
    ]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    response = generate(model, tokenizer, prompt=prompt, verbose=False)
    return evaluator.evaluate_alignments(response, correct)

In [8]:
from tqdm import tqdm

for prompt, correct in tqdm(zip(all_prompts, all_correct)):
    all_results[(20, 3)].extend(evaluate(prompt, correct))
print("Average weighted accuracy: ", sum([r[-1].weighted_accuracy for r in all_results[(20, 3)]])/len(all_results[(20, 3)]))
print("Average F1 score: ", sum([r[-1].f1 for r in all_results[(20, 3)]])/len(all_results[(20, 3)]))

20it [03:26, 10.31s/it]


TypeError: 'int' object is not subscriptable

In [10]:
all_results[(20, 3)][0]

3